In [1]:
import sys, os, random
sys.path.insert(0,os.path.expanduser('~/git/pyzx')) # git version
sys.path.insert(0,'/workspaces/pyzx')
import pyzx as zx
from pyzx.pauliweb import preprocess, compute_pauli_webs

This notebook demonstrates using PyZX's simplifier to construct the reduced ZX diagram of a random Clifford+T circuit, then using PyZX's gflow-finding algorithm to automatically associate each node `n` in the diagram to an integer `order[n]` and a Pauli web (a.k.a. _correlation surface_) `web[n]` with the following properties:

1. the boundary of `web[n]` consists of `n` and inputs
2. `web[n]` connects to `n` by a single edge, which is the same colour as `n`
3. for non-Clifford nodes `m, n`, if `m` is in `web[n].vertices()` then `order[m] < order[n]`

The values of `web[n]` at non-Clifford nodes and at outputs should be enough data to deterministically implement the diagram via lattice surgery and either measure at the end or compute the updated Pauli frame.

In [2]:
# Generate a random CNOT, H, T circuit
random.seed(1330)
c = zx.generate.CNOT_HAD_PHASE_circuit(qubits=3, depth=50)
zx.draw(c)

In [3]:
# Convert to a ZX diagram and call the full_reduce procedure on it (PyZX's main ZX diagram optimisation pass)
g = c.to_graph()
zx.full_reduce(g)

# Normalise compacts the circuit visually and ensures every input/output is connected to a Z spider
g.normalize()

# For simplicity, preprocess strips off any local Clifford unitaries and saves them in in_circ and out_circ,
# then introduces a dummy Z and X node at every output, so we can compute Z- and X-bounded Pauli webs there
in_circ, out_circ = preprocess(g)

# Compute the time-ordering on nodes (which is only important for the non-Clifford nodes) and compute the Pauli
# webs for every node.
order, webs = compute_pauli_webs(g)

# Draw the simplified ZX diagram. Note blue edges correspond to edges with Hadamard gates
zx.draw(g, labels=True)

In [4]:
# Once the Pauli webs have been computed, a specific web can be highlighted by `zx.draw` by passing it in as
# an optional argument. Note that webs change color when they cross Hadamard edges.
zx.draw(g, labels=True, pauli_web=webs[87])

In [5]:
c = zx.qasm("""
qreg q[1];
t q[0];
""")
zx.draw(c)

In [6]:
g = c.to_graph()
zx.full_reduce(g)
in_circ, out_circ = preprocess(g)
order, webs = compute_pauli_webs(g)
print(f'{in_circ.gates} {out_circ.gates}')
zx.draw(g)

[] []
